# Packages


In [11]:
import os, requests
import numpy as np
import matplotlib.pyplot as plt 
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive/') 


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
!ls "/content/drive/"
%cd '/content/drive/My Drive/NMA_EasyCabbage'

'My Drive'
/content/drive/.shortcut-targets-by-id/1Mz7APXhT0fsT11lLYjrD_MdzQEYZzpMw/NMA_EasyCabbage


# Data retrieval


In [4]:
# get file names
fname = []
for j in range(3):
  fname.append('steinmetz_part%d.npz'%j)
fname

['steinmetz_part0.npz', 'steinmetz_part1.npz', 'steinmetz_part2.npz']

In [5]:
# if nessescary: download data (now stored on the drive)
def download_data(fname):
  """
  This function downloads the data required datasets from the Steinmetz website

  Args:
      fname (list of str): filenames of required datasets

  """
  url = ["https://osf.io/agvxh/download"]
  url.append("https://osf.io/uv3mw/download")
  url.append("https://osf.io/ehmw2/download")
  for j in range(len(url)):
    if not os.path.isfile(fname[j]):
      try:
        r = requests.get(url[j])
      except requests.ConnectionError:
        print("!!! Failed to download data !!!")
      else:
        if r.status_code != requests.codes.ok:
          print("!!! Failed to download data !!!")
        else:
          with open(fname[j], "wb") as fid:
            fid.write(r.content)

download_data(fname)

In [6]:
def load_data(fname):
  """
  This function loads data and stacks them horiontally

  Args:
      fname (list of str): file names of the required datasets

  Returns:
      (np.array): steinmetz data
  """
  alldat = np.array([])
  for j in range(len(fname)):
    alldat = np.hstack((alldat, np.load('steinmetz_part%d.npz'%j, allow_pickle=True)['dat']))

  return alldat

all_dat = load_data(fname)

# Constants

In [7]:
N_TIMEPOINTS = all_dat[0]['spks'].shape[-1]
BIN_SIZE_DATA = all_dat[0]['bin_size']
MAX_INTERVAL = N_TIMEPOINTS*BIN_SIZE_DATA

for neurons in all_dat:
  print(neurons['spks'].shape[1])

214
251
228
249
254
290
252
250
372
447
342
340
300
268
404
280
224
316
247
235
124
444
151
187
261
178
253
142
128
143
237
260
191
296
311
258
181
199
343


# Variable settings

In [8]:
# tested_bin_size
bin_size_ISI = 0.05

# Helper functions

In [9]:
def normalize(number_array):
  n = float(np.sum(number_array))
  if not n:
    return number_array, n
  return number_array/n, n

def get_ISI_trial(spike):
  spike_times = np.array([])
  for t,s in enumerate(spike):
    for i in range(s):
      spike_times = np.hstack([spike_times, [t]])


  spike_intervals = np.diff(spike_times) * BIN_SIZE_DATA
  return spike_intervals

def get_ISI_neuron(neuron, trials):
  spike_intervals = np.array([])
  
  for trial in trials:
    spike = neuron[trial]
    spike_interval = get_ISI_trial(spike)
    spike_intervals = np.hstack([spike_intervals, spike_interval])
  return spike_intervals

def get_bins(neuron, bin_size_ISI, trials):
  spike_intervals = get_ISI_neuron(neuron, trials)
  spikes_per_bin, bins = np.histogram(spike_intervals, bins = np.arange(0, MAX_INTERVAL, bin_size_ISI))
  spikes_per_bin, n_spikes = normalize(spikes_per_bin)
  return spikes_per_bin, n_spikes


def get_n_dimensional_neurons(neurons, bin_size_ISI):
  neuron_amount = neurons.shape[0]
  n_bins = int(MAX_INTERVAL/bin_size_ISI) - 1
  n_dimensional_neurons = np.empty((neuron_amount, n_bins))
  rel_spikes = np.empty((neuron_amount,1))
  trials = [i for i in range(neurons.shape[1])]
  for i in range(neuron_amount):
    n_dimensional_neurons[i,:], rel_spikes[i,0] = get_bins(neurons[i], bin_size_ISI, trials)

  rel_spikes, total_spikes = normalize(rel_spikes)
  n_dimensional_neurons = np.hstack([n_dimensional_neurons, rel_spikes])

  return n_dimensional_neurons

In [13]:
n_dim_bins_seperate_sessions = []

for i,session in enumerate(all_dat):
  neurons = session['spks']
  pca_input = get_n_dimensional_neurons(neurons, bin_size_ISI)
  print(pca_input)
  n_dim_bins_seperate_sessions.append(pca_input)

  with open(f'bins_session{i}_b{bin_size_ISI}.pkl', 'wb') as f:
    pickle.dump(pca_input, f)

  #np.save(f'bins_session{i}_b{bin_size_ISI}.npy', pca_input)

[[2.10526316e-01 4.21052632e-01 5.26315789e-02 ... 0.00000000e+00
  0.00000000e+00 1.49013297e-05]
 [1.20535714e-01 1.07142857e-01 1.96428571e-01 ... 0.00000000e+00
  0.00000000e+00 1.75678834e-04]
 [3.89534884e-01 2.60174419e-01 1.26453488e-01 ... 0.00000000e+00
  0.00000000e+00 5.39584990e-04]
 ...
 [1.71232877e-01 1.36986301e-01 1.78082192e-01 ... 0.00000000e+00
  0.00000000e+00 1.14504954e-04]
 [5.00000000e-01 0.00000000e+00 5.00000000e-01 ... 0.00000000e+00
  0.00000000e+00 1.56856102e-06]
 [1.75675676e-01 1.01351351e-01 9.79729730e-02 ... 0.00000000e+00
  0.00000000e+00 2.32147031e-04]]
[[1.62637363e-01 1.91648352e-01 1.74065934e-01 ... 0.00000000e+00
  0.00000000e+00 1.24677688e-03]
 [2.24242424e-01 2.02525253e-01 1.54040404e-01 ... 0.00000000e+00
  0.00000000e+00 1.08510691e-03]
 [1.51062156e-01 2.19118804e-01 1.69944925e-01 ... 0.00000000e+00
  0.00000000e+00 1.39310190e-03]
 ...
 [1.36431784e-01 1.64167916e-01 2.69490255e-01 ... 0.00000000e+00
  0.00000000e+00 1.46215416e-03]

In [ ]:
print(type(pca_input))